## Scenario/Stakeholder Based Analysis of NYC taxi rides data
##### Authors: Panini Mokrala, Dmitrii Danilov

In [1]:
!pip install PyShp
!pip install socrata-py

import io
import zipfile
import pandas as pd
import shapefile
import requests
from shapely.geometry import Polygon

     |████████████████████████████████| 225kB 9.5MB/s 
  Created wheel for PyShp: filename=pyshp-2.1.2-cp36-none-any.whl size=36216 sha256=349949cef5b402535df8738ee160d480cb8d6b46905fef23169afbaad36b99aa
  Stored in directory: /root/.cache/pip/wheels/96/6c/53/4112475adf3b831da97f083163d0f38ee6daac9c1b13f7afea
Successfully built PyShp
  Created wheel for socrata-py: filename=socrata_py-1.0.5-cp36-none-any.whl size=24489 sha256=de244848c148c5bfbf1e05b52ab72636acaf7b586588a5fdc13354c6ade81381
  Stored in directory: /root/.cache/pip/wheels/01/d2/b8/f13567530a071afda2644925360b36a0e5370cecf0b03414b2
Successfully built socrata-py


In [2]:
def import_taxi_zones():
    taxi_zones = requests.get('https://s3.amazonaws.com/nyc-tlc/misc/taxi_zones.zip')
    with zipfile.ZipFile(io.BytesIO(taxi_zones.content), 'r') as myzip:
        sf = shapefile.Reader(shp=myzip.open('taxi_zones.shp'),
                              shx=myzip.open('taxi_zones.shx'),
                              dbf=myzip.open('taxi_zones.dbf'))

    fields = [x[0].lower() for x in sf.fields][1:]
    records = sf.records()
    shps = [Polygon(s.points).wkt for s in sf.shapes()]
    df = pd.DataFrame(columns=fields, data=records)
    df = df.assign(coords=shps)
    return df

In [4]:
taxi_zones = import_taxi_zones()
taxi_zones.head()

,objectid,shape_leng,shape_area,zone,locationid,borough,coords
0,1,0.116357,0.000782,Newark Airport,1,EWR,"POLYGON ((933100.9183527103 192536.0856972019,..."
1,2,0.433470,0.004866,Jamaica Bay,2,Queens,"POLYGON ((1033269.243591294 172126.0078125, 10..."
2,3,0.084341,0.000314,Allerton/Pelham Gardens,3,Bronx,"POLYGON ((1026308.769506663 256767.6975403726,..."
3,4,0.043567,0.000112,Alphabet City,4,Manhattan,"POLYGON ((992073.4667968601 203714.0759887695,..."
4,5,0.092146,0.000498,Arden Heights,5,Staten Island,"POLYGON ((935843.3104932606 144283.335850656, ..."
